In [2]:
import json
import unidecode
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output
import numpy as np

import textdistance as td

### Utils

In [6]:
def ler_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# transoformar em minusculo
# remover palavras repetidas
def normalizar_palavra(palavra):
    palavra_normalizada = unidecode.unidecode(palavra.lower().replace(",","").replace('"',"").replace(".",""))
    return palavra_normalizada

def obterPalavrasTexto(texto:str,repetirPalavras = False):
    texto_separado = texto.split()
    palavras_do_texto = []
    for palavra in texto_separado:
        palavraNormalizada = normalizar_palavra(palavra)
        if not repetirPalavras and palavraNormalizada not in palavras_do_texto:
            palavras_do_texto.append(palavraNormalizada)   
    return palavras_do_texto
        
# Similaridade usando coseno
def cosine_similarity_text(text1, text2):
    # Criar vetorizador TF-IDF
    vectorizer = TfidfVectorizer()
    
    # Ajustar e transformar os textos para a matriz TF-IDF
    tfidf_matrix = vectorizer.fit_transform([text1, text2])
    
    # Calcular a similaridade cosseno entre os textos
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    
    return cosine_sim

#Similaridade por distancia entre textos:
def distance_similarity_text(text1,text2):
    return td.levenshtein.normalized_similarity(text1,text2)

## Filtrando textos similares obtidos com função SequenceMatcher

In [3]:
similaresTest =  ler_json("./similaresGrande.json") # Sem Seleção manual

In [5]:
linhas = [
    0, 1, 17, 34, 37, 50, 54, 94, # Diff
    2, 4, 25, 32, 33, 91, 197, 213 # Similar
]

def cria_texto_objeto(texto, linha, index, pos):
    return {
        "url": texto[linha][pos]["url"],
        "titulo": texto[linha][pos]["titulo"],
        "titulo-limpo": texto[linha][pos]["tituloLimpo"],
        "texto-limpo": texto[linha][pos]["textoMinusculo"],
        "similar": False if index < len(linhas) / 2 else True
    }

textosSimilaresFiltrados = []
for index,linha in enumerate(linhas):
    textosSimilaresFiltrados.append([
        cria_texto_objeto(similaresTest,linha,index,0),
        cria_texto_objeto(similaresTest,linha,index,1),  
    ])


In [6]:
with open(f"./similares.json",'w+',encoding="utf8") as f:
    json.dump(textosSimilaresFiltrados,f,indent=3,allow_nan=True,ensure_ascii = False)
    f.close()

## Descobrindo limite de similaridade

In [12]:
LIMITE_SIMILARIDADE = 0.21073081490174875 #+ 0.02

def ehSimilar(grau_similaridade):
    return True if grau_similaridade > LIMITE_SIMILARIDADE else False

## Avaliando similaridade dos textos

In [13]:
textosSimilares = ler_json("./similares.json")

for grupoTexto in textosSimilares:
    texto1,texto2 = grupoTexto[0]["titulo-limpo"], grupoTexto[1]["titulo-limpo"]
    
    similaridadeCosine = cosine_similarity_text(
        " ".join(obterPalavrasTexto(texto1)),
        " ".join(obterPalavrasTexto(texto2)),
    )
    
    similaridadeDistance =distance_similarity_text(
        "".join(obterPalavrasTexto(texto1)),
        "".join(obterPalavrasTexto(texto2)),
    )
    
    print(f"{grupoTexto[0]['similar']}: Cosine: {similaridadeCosine} - Distance: {similaridadeDistance} == {ehSimilar(similaridadeCosine)}")

False: Cosine: 0.07397129483465995 - Distance: 0.42000000000000004 == False
False: Cosine: 0.07397129483465995 - Distance: 0.36 == False
False: Cosine: 0.20199309249791836 - Distance: 0.37209302325581395 == False
False: Cosine: 0.06804963087832473 - Distance: 0.40384615384615385 == False
False: Cosine: 0.11234277891542778 - Distance: 0.39682539682539686 == False
False: Cosine: 0.21528271546341893 - Distance: 0.45679012345679015 == True
False: Cosine: 0.38087260847594373 - Distance: 0.46031746031746035 == True
False: Cosine: 0.18443191662261307 - Distance: 0.4117647058823529 == False
True: Cosine: 0.24478531173455215 - Distance: 0.509090909090909 == True
True: Cosine: 0.5023287782256718 - Distance: 0.4516129032258065 == True
True: Cosine: 0.40711465268091374 - Distance: 0.4666666666666667 == True
True: Cosine: 0.2532094495161746 - Distance: 0.5660377358490566 == True
True: Cosine: 0.22150013430590976 - Distance: 0.42105263157894735 == True
True: Cosine: 0.615218690896332 - Distance: 0.7

In [4]:


naoSimilariadeMedia = 0
similariadeMedia = 0
acertosFalse = 0
acertosTrue = 0

for grupoTexto in textosSimilares:
    texto1,texto2 = grupoTexto[0]["titulo"], grupoTexto[1]["titulo"]
    
    similaridade = cosine_similarity_text(
        " ".join(obterPalavrasTexto(texto1)),
        " ".join(obterPalavrasTexto(texto2)),
    )
    
    if grupoTexto[0]["similar"] == False:
        naoSimilariadeMedia += similaridade
        if ehSimilar(similaridade) == grupoTexto[0]['similar']:
            acertosFalse += 1
    else:
        similariadeMedia += similaridade
        if ehSimilar(similaridade) == grupoTexto[0]['similar']:
            acertosTrue += 1
    
    #print(f"Similaridade: {similaridade}; Previsão: {ehSimilar(similaridade)}; Real: {grupoTexto[0]['similar']}")

metade = len(textosSimilares)/2
print(f"Acertos Similares: {acertosTrue}/{metade}")
print(f"Acertos Não Similares: {acertosFalse}/{metade}")
print(f"Acertos Totais: {acertosFalse + acertosTrue}/{len(textosSimilares)} = {acertosFalse + acertosTrue/len(textosSimilares)*100}%")
print(f"Similaridade Média: {similariadeMedia/metade}")
print(f"Não Similaridade Média: {naoSimilariadeMedia/metade}")

# Teste
# Acertos Similares: 7/8.0
# Acertos Não Similares: 5/8.0
# Acertos Totais: 12/16 = 48.75%
# Similaridade Média: 0.34594763540328094
# Não Similaridade Média: 0.21073081490174875
    


Acertos Similares: 7/8.0
Acertos Não Similares: 5/8.0
Acertos Totais: 12/16 = 48.75%
Similaridade Média: 0.34594763540328094
Não Similaridade Média: 0.21073081490174875


# Pesquisando nos textos obtidos

In [8]:
fatooufake = ler_json("./FatoouFake/fatofakeTeste1.json")
boatosorg = ler_json("./Boatos.org/boatosorg_ref.json")

In [9]:
textosRepetidos = []
maximoTextos = int(len(boatosorg)*0.1)

for index,texto_b in enumerate(boatosorg[0:maximoTextos]):
    for texto_ff in fatooufake:
        
        texto1,texto2 = texto_b["texto"], texto_ff["titulo"]
        similaridade = cosine_similarity_text(
            " ".join(obterPalavrasTexto(texto1)),
            " ".join(obterPalavrasTexto(texto2)),
        )
        
        if ehSimilar(similaridade):
            textosRepetidos.append([
                texto_b, texto_ff
            ])
            textosObservados = 0
            break # Se achou um similar não precisa de procurar mais
    textosObservados = 0


In [11]:
similaridadeTeste = cosine_similarity_text(
    " ".join(obterPalavrasTexto("Não há provas e nem indícios de que destruição de lavoura de café em Migrantinópolis seja obra do MST")),
    " ".join(obterPalavrasTexto("É #FAKE que vídeo mostre obra do governo Bolsonaro em ferrovia de Marabá"))
)
print(similaridadeTeste,ehSimilar(similaridadeTeste))

0.22288823689910964 False


In [10]:
with open(f"./similaresLimite3.json",'w+',encoding="utf8") as f:
    json.dump(textosRepetidos,f,indent=3,allow_nan=True,ensure_ascii = False)
    f.close()